# CM3005 Data Science Coursework
# Linear Regression Project Starter Notebook

# 1. Domain-Specific Area and Objectives

## Domain-Specific Area



## Objectives
1. To build a model that predicts [target variable].
2. To identify key trends and insights in the dataset.
3. To evaluate the model's performance and discuss its implications.

In [898]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [899]:
# Initialise the year variable to use in all the datasets
year = 2019

# 2. Datasets Initialization

## 2.1 Dataset Description

### 2.1.1 Resale Flat Price Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [900]:
# Load the Dataset
try:
    resaleFlatPrice = pd.read_csv('./data/resaleFlatPrices.csv')
    # Display information about the dataset
    print(resaleFlatPrice.head())
    print("")
    print(resaleFlatPrice.info())
    print("")
    print("Shape", resaleFlatPrice.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

     month        town flat_type block        street_name storey_range  \
0  2017-01  ANG MO KIO    2 ROOM   406  ANG MO KIO AVE 10     10 TO 12   
1  2017-01  ANG MO KIO    3 ROOM   108   ANG MO KIO AVE 4     01 TO 03   
2  2017-01  ANG MO KIO    3 ROOM   602   ANG MO KIO AVE 5     01 TO 03   
3  2017-01  ANG MO KIO    3 ROOM   465  ANG MO KIO AVE 10     04 TO 06   
4  2017-01  ANG MO KIO    3 ROOM   601   ANG MO KIO AVE 5     01 TO 03   

   floor_area_sqm      flat_model  lease_commence_date     remaining_lease  \
0            44.0        Improved                 1979  61 years 04 months   
1            67.0  New Generation                 1978  60 years 07 months   
2            67.0  New Generation                 1980  62 years 05 months   
3            68.0  New Generation                 1980   62 years 01 month   
4            67.0  New Generation                 1980  62 years 05 months   

   resale_price  
0      232000.0  
1      250000.0  
2      262000.0  
3      265000.

### 2.1.2 CPI Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [901]:
# Load the Dataset
try:
    cpi = pd.read_csv('./data/CPI.csv')
    # Display information about the dataset
    print(cpi.head())
    print("")
    print("Shape", cpi.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

                            DataSeries  2024Oct  2024Sep  2024Aug  2024Jul  \
0                            All Items  117.058  116.864  116.570  116.328   
1                                 Food  119.264  118.886  118.666  118.303   
2      Food Excl Food Serving Services  117.907  117.605  117.096  116.512   
3                      Bread & Cereals  119.428  118.818  118.120  118.209   
4                                 Meat  123.758  124.054  123.747  122.224   

   2024Jun  2024May  2024Apr  2024Mar  2024Feb  ...  1961Oct  1961Sep  \
0  116.065  116.316  116.363  115.406  115.449  ...   24.527   24.455   
1  118.108  117.951  117.517  117.499  117.787  ...   21.701   21.549   
2  116.601  116.696  116.031  116.312  117.559  ...       na       na   
3  117.849  117.711  117.152  116.644  116.501  ...       na       na   
4  122.779  123.043  123.372  124.619  124.916  ...       na       na   

   1961Aug  1961Jul  1961Jun  1961May  1961Apr  1961Mar  1961Feb  1961Jan  
0   24.551   24.

### 2.1.3 Interest Rates Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [902]:
# Load the Dataset
try:
    interestRate = pd.read_csv('./data/banksInterestRates.csv')
    print(interestRate.head())
    print("")
    print(interestRate.info())
    print("")
    print("Shape", interestRate.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

                                          DataSeries  2024Nov  2024Oct  \
0          Government Securities - 5-Year Bond Yield     2.73     2.71   
1          Government Securities - 2-Year Bond Yield     2.77     2.69   
2         Government Securities - 10-Year Bond Yield     2.74     2.82   
3  Government Securities - 1-Year Treasury Bills ...     2.75     2.73   
4         Government Securities - 15-Year Bond Yield     2.80     2.87   

   2024Sep  2024Aug  2024Jul  2024Jun  2024May  2024Apr  2024Mar  ...  \
0     2.50     2.56     2.80     3.19     3.34     3.40     3.05  ...   
1     2.44     2.55     2.94     3.34     3.42     3.50     3.36  ...   
2     2.62     2.71     2.88     3.21     3.36     3.45     3.11  ...   
3     3.04     3.14     3.39     3.58     3.57     3.57     3.46  ...   
4     2.69     2.79     2.90     3.21     3.34     3.45     3.09  ...   

   1988Oct  1988Sep  1988Aug  1988Jul  1988Jun  1988May  1988Apr  1988Mar  \
0     5.38     5.62     5.74     5.32  

### 2.1.4 Salary Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [903]:
# Load the Dataset
try:
    salary = pd.read_csv('./data/medianIncome.csv')
    print(salary.head())
    print("")
    print(salary.info())
    print("")
    print("Shape", salary.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

   year     sex  med_income_incl_empcpf  med_income_excl_empcpf
0  2001    male                    2514                    2250
1  2001  female                    2204                    1950
2  2002    male                    2514                    2176
3  2002  female                    2219                    1950
4  2003    male                    2514                    2217

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   year                    42 non-null     int64 
 1   sex                     42 non-null     object
 2   med_income_incl_empcpf  42 non-null     int64 
 3   med_income_excl_empcpf  42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB
None

Shape (42, 4)


### 2.1.5 Employment Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [904]:
# Load the Dataset
try:
    employmentRate = pd.read_csv('./data/employmentRate.csv')
    print(employmentRate.head())
    print("")
    print(employmentRate.info())
    print("")
    print("Shape", employmentRate.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

   year     sex  emp_rate
0  1991    male      90.9
1  1991  female      50.8
2  1992    male      90.0
3  1992  female      51.9
4  1993    male      90.3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      60 non-null     int64  
 1   sex       60 non-null     object 
 2   emp_rate  60 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.5+ KB
None

Shape (60, 3)


### 2.1.6 Population Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [905]:
# Load the Dataset
try:
    population = pd.read_csv('./data/population.csv')
    print(population.head())
    print("")
    print(population.info())
    print("")
    print("Shape", population.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

   Data Series  Total Population (Number)  \
0         2024                    6036860   
1         2023                    5917648   
2         2022                    5637022   
3         2021                    5453566   
4         2020                    5685807   

   Total Population -> 0 - 14 Years (Number)  \
0                                     677425   
1                                     681459   
2                                     675638   
3                                     670531   
4                                     688984   

   Total Population -> 15 - 64 Years (Number)  \
0                                     4530831   
1                                     4451242   
2                                     4226527   
3                                     4087584   
4                                     4320933   

   Total Population -> 65 Years & Over (Number)  \
0                                        828604   
1                                        78

### 2.1.7 Schools Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [906]:
# Load the Dataset
try:
    schools = pd.read_csv('./data/schools.csv')
    print(schools.head())
    print("")
    print(schools.info())
    print("")
    print("Shape", schools.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

                      school_name                            url_address  \
0        ADMIRALTY PRIMARY SCHOOL       https://admiraltypri.moe.edu.sg/   
1      ADMIRALTY SECONDARY SCHOOL     http://www.admiraltysec.moe.edu.sg   
2    AHMAD IBRAHIM PRIMARY SCHOOL  http://www.ahmadibrahimpri.moe.edu.sg   
3  AHMAD IBRAHIM SECONDARY SCHOOL  http://www.ahmadibrahimsec.moe.edu.sg   
4                  AI TONG SCHOOL           http://www.aitong.moe.edu.sg   

                   address  postal_code telephone_no telephone_no_2    fax_no  \
0    11   WOODLANDS CIRCLE       738907     63620598             na  63627512   
1  31   WOODLANDS CRESCENT       737916     63651733       63654596  63652774   
2    10   YISHUN STREET 11       768643     67592906             na  67592927   
3     751  YISHUN AVENUE 7       768928     67585384             na  67557778   
4   100  Bright Hill Drive       579646     64547672             na  64532726   

  fax_no_2            email_address                  mrt

### 2.1.8 Hawker Centres Dataset Description
- Source: [Mention dataset source, e.g., data.gov.sg]
- Size: [Number of entries and columns]
- Structure: [Types of data, e.g., numeric, categorical]
- Relevance: [Why this dataset is suitable for the project]

In [907]:
# Load the Dataset
try:
    hawkerCentres = pd.read_csv('./data/hawkerCentres.csv')
    print(hawkerCentres.head())
    print("")
    print(hawkerCentres.info())
    print("")
    print("Shape", hawkerCentres.shape)
except FileNotFoundError:
    print("Dataset not found. Please ensure the file is in the correct location.")

            name_of_centre                                 location_of_centre  \
0    Adam Road Food Centre                            2, Adam Road, S(289876)   
1  Amoy Street Food Centre  National Development Building, Annex B, Telok ...   
2        Bedok Food Centre                           1, Bedok Road, S(469572)   
3      Beo Crescent Market                       38A, Beo Crescent, S(169982)   
4       Berseh Food Centre                        166, Jalan Besar, S(208877)   

  type_of_centre       owner  no_of_stalls  no_of_cooked_food_stalls  \
0             HC  Government            32                        32   
1             HC  Government           135                       134   
2             HC  Government            32                        32   
3            MHC  Government            94                        32   
4             HC  Government            66                        66   

   no_of_mkt_produce_stalls  
0                         0  
1                   

# 3. Data Cleaning and Preparation

Steps:
1. Check for missing values in all datasets.
2. Drop irrelevant columns.
3. Handle missing data.
4. Normalize numerical features.

## 3.1: Check for missing values in all datasets

In [908]:
# Combine all datasets into a dictionary
datasets = {
    "Resale Flat Prices": resaleFlatPrice,
    "CPI": cpi,
    "Interest Rates": interestRate,
    "Median Income": salary,
    "Employment Rate": employmentRate,
    "Population": population,
    "Schools": schools,
    "Hawker Centres": hawkerCentres
}

In [909]:
# Check for null or NA values
print("Checking for null or NA values in datasets:")
for name, dataset in datasets.items():
    print(f"\n{name}:")
    if dataset.isnull().sum().sum() > 0:
        print(dataset.isnull().sum())
    else:
        print("No null or NA values found.")

Checking for null or NA values in datasets:

Resale Flat Prices:
No null or NA values found.

CPI:
No null or NA values found.

Interest Rates:
No null or NA values found.

Median Income:
No null or NA values found.

Employment Rate:
No null or NA values found.

Population:
No null or NA values found.

Schools:
school_name             0
url_address             0
address                 0
postal_code             0
telephone_no            0
telephone_no_2          0
fax_no                  0
fax_no_2                0
email_address           0
mrt_desc                0
bus_desc                0
principal_name          0
first_vp_name           4
second_vp_name         10
third_vp_name         216
fourth_vp_name        332
fifth_vp_name         334
sixth_vp_name         337
dgp_code                0
zone_code               0
type_code               0
nature_code             0
session_code            0
mainlevel_code          0
sap_ind                 0
autonomous_ind          0
gifted_ind 

## 3.2: Drop irrelevant columns and rows

### 3.2.1: Resale Flat Price Dataset

- Combine 'block' and 'street_name' columns into a single 'address' column.
- Drop 'block' and 'street_name' columns.
- Drop rows with timestamp before 2019.

In [910]:
# Check if the necessary columns exist
if 'block' in resaleFlatPrice.columns and 'street_name' in resaleFlatPrice.columns:
    # Combine 'block' and 'street_name' columns into a single column
    resaleFlatPrice['address'] = resaleFlatPrice['block'] + ' ' + resaleFlatPrice['street_name']

    # Drop the 'block' and 'street_name' columns
    resaleFlatPrice.drop(columns=['block', 'street_name'], inplace=True)

# Display the updated DataFrame
resaleFlatPrice.head()


,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2017-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,601 ANG MO KIO AVE 5


In [911]:
# Ensure 'month' column is in datetime format
resaleFlatPrice['month'] = pd.to_datetime(resaleFlatPrice['month'], format='%Y-%m')

# Sort the DataFrame in descending order of the 'month' column
resaleFlatPrice.sort_values(by='month', ascending=False, inplace=True)

# Filter and keep only rows where the year is greater than or equal to the specified year
resaleFlatPrice = resaleFlatPrice[resaleFlatPrice['month'].dt.year >= year]

# Reset the index to 0
resaleFlatPrice.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
print(resaleFlatPrice.head())
print("")
print("Shape:", resaleFlatPrice.shape)

       month         town flat_type storey_range  floor_area_sqm  \
0 2024-12-01  JURONG WEST    5 ROOM     10 TO 12           114.0   
1 2024-12-01      PUNGGOL    5 ROOM     10 TO 12           113.0   
2 2024-12-01      PUNGGOL    5 ROOM     04 TO 06           111.0   
3 2024-12-01  BUKIT BATOK    3 ROOM     04 TO 06            73.0   
4 2024-12-01  BUKIT BATOK    3 ROOM     07 TO 09            74.0   

          flat_model  lease_commence_date     remaining_lease  resale_price  \
0  Premium Apartment                 2008  82 years 09 months      620000.0   
1           Improved                 2018            93 years      775000.0   
2  Premium Apartment                 2013  87 years 04 months      850000.0   
3            Model A                 1984  59 years 02 months      428000.0   
4            Model A                 1985  59 years 03 months      425888.0   

                  address  
0  989A JURONG WEST ST 93  
1        231B SUMANG LANE  
2         289C PUNGGOL PL  
3   

### 3.2.2: CPI Dataset

TODOs:
- Drop all rows except All Items.
- Tranpose the dataset and rename the columns to 'month' and 'cpi'.
- Drop the column 'DataSeries'.
- Drop rows from 'month' column with date and time before the specified year above.

In [912]:
# Filter the dataset to keep only rows where the 'DataSeries' is 'All Items'
cpi = cpi[cpi['DataSeries'] == 'All Items']

# Transform the data from wide to long format
cpi = cpi.melt(id_vars=['DataSeries'], var_name='month', value_name='cpi')

cpi.head()

,DataSeries,month,cpi
0,All Items,2024Oct,117.058
1,All Items,2024Sep,116.864
2,All Items,2024Aug,116.57
3,All Items,2024Jul,116.328
4,All Items,2024Jun,116.065


In [913]:
try:
    # Drop the 'DataSeries' column and rename 'CPI_Value' to 'CPI'
    if 'DataSeries' in cpi.columns:
        cpi.drop(columns=['DataSeries'], inplace=True)
        print("Dropped 'DataSeries' column.")
    else:
        print("'DataSeries' column has already been dropped.")

    # Display the updated DataFrame
    print("")
    print(cpi.head())
except Exception as e:
    print(f"An error occurred: {e}")

Dropped 'DataSeries' column.

     month      cpi
0  2024Oct  117.058
1  2024Sep  116.864
2  2024Aug   116.57
3  2024Jul  116.328
4  2024Jun  116.065


In [914]:
try:
    # Ensure 'month' column is in datetime format
    cpi['month'] = pd.to_datetime(cpi['month'], format='%Y%b', errors='coerce')
    
    # Drop rows with invalid or missing dates
    cpi.dropna(subset=['month'], inplace=True)

    # Filter and keep only rows where the year is greater than or equal to the specified year
    cpi = cpi[cpi['month'].dt.year >= year]

    # Reset the index to start from 0
    cpi.reset_index(drop=True, inplace=True)

    # Display the updated DataFrame and its shape
    print(cpi.head())
    print("")
    print("Shape:", cpi.shape)
except Exception as e:
    print(f"An error occurred: {e}")

       month      cpi
0 2024-10-01  117.058
1 2024-09-01  116.864
2 2024-08-01   116.57
3 2024-07-01  116.328
4 2024-06-01  116.065

Shape: (70, 2)


### 3.2.3: Interest Rates Dataset

TODOs:
- Drop all rows except Singapore Overnight Rate Average.
- Tranpose the dataset and rename the columns to 'month' and 'interest_rate'.
- Drop the column 'DataSeries'.
- Drop rows from 'month' column with date and time before the specified year above.

In [915]:
# Filter the dataset to keep only rows where the 'DataSeries' is 'All Items'
interestRate = interestRate[interestRate['DataSeries'] == 'Singapore Overnight Rate Average']

# Transform the data from wide to long format
interestRate = interestRate.melt(id_vars=['DataSeries'], var_name='month', value_name='interest_rate')

interestRate.head()

,DataSeries,month,interest_rate
0,Singapore Overnight Rate Average,2024Nov,3.0839
1,Singapore Overnight Rate Average,2024Oct,3.1559
2,Singapore Overnight Rate Average,2024Sep,3.8808
3,Singapore Overnight Rate Average,2024Aug,3.5355
4,Singapore Overnight Rate Average,2024Jul,3.4336


In [916]:
try:
    # Drop the 'DataSeries' column and rename 'CPI_Value' to 'CPI'
    if 'DataSeries' in interestRate.columns:
        interestRate.drop(columns=['DataSeries'], inplace=True)
        print("Dropped 'DataSeries' column.")
    else:
        print("'DataSeries' column has already been dropped.")

    # Display the updated DataFrame
    print("")
    print(interestRate.head())
except Exception as e:
    print(f"An error occurred: {e}")

Dropped 'DataSeries' column.

     month interest_rate
0  2024Nov        3.0839
1  2024Oct        3.1559
2  2024Sep        3.8808
3  2024Aug        3.5355
4  2024Jul        3.4336


In [917]:
try:
    # Ensure 'month' column is in datetime format
    interestRate['month'] = pd.to_datetime(interestRate['month'], format='%Y%b', errors='coerce')
    
    # Drop rows with invalid or missing dates
    interestRate.dropna(subset=['month'], inplace=True)

    # Filter and keep only rows where the year is greater than or equal to the specified year
    interestRate = interestRate[interestRate['month'].dt.year >= year]

    # Reset the index to start from 0
    interestRate.reset_index(drop=True, inplace=True)

    # Display the updated DataFrame and its shape
    print(interestRate.head())
    print("")
    print("Shape:", interestRate.shape)
except Exception as e:
    print(f"An error occurred: {e}")

       month interest_rate
0 2024-11-01        3.0839
1 2024-10-01        3.1559
2 2024-09-01        3.8808
3 2024-08-01        3.5355
4 2024-07-01        3.4336

Shape: (71, 2)


### 3.2.4: Salary Dataset

TODOs:
- Drop the med_income_excl_empcpf column.

In [918]:
# Modify the DataFrame
try:
    # Remove the 'med_income_excl_empcpf' column
    salary.drop(columns=['med_income_excl_empcpf'], inplace=True)
    
    # Rename the 'year' column to 'month'
    salary.rename(columns={'year': 'month'}, inplace=True)
    
    # Convert 'month' column to datetime format with a placeholder month (e.g., January)
    salary['month'] = pd.to_datetime(salary['month'].astype(str) + '-01', format='%Y-%m')

    
    
    # Display the updated DataFrame
    print(salary.head())
except Exception as e:
    print(f"An error occurred: {e}")

    

       month     sex  med_income_incl_empcpf
0 2001-01-01    male                    2514
1 2001-01-01  female                    2204
2 2002-01-01    male                    2514
3 2002-01-01  female                    2219
4 2003-01-01    male                    2514


In [919]:
try:
    # Group by 'month' and calculate the average income
    salary = salary.groupby('month', as_index=False).agg({'med_income_incl_empcpf': 'mean'})
    
    # Rename the column
    salary.rename(columns={'med_income_incl_empcpf': 'median_income'}, inplace=True)

     # Sort by 'month' in descending order
    salary.sort_values(by='month', ascending=False, inplace=True)

    # Reset index
    salary.reset_index(drop=True, inplace=True)

   # Display the updated DataFrame
    print(salary.head())
except Exception as e:
    print(f"An error occurred: {e}")

       month  median_income
0 2022-01-01         5199.0
1 2021-01-01         4656.0
2 2020-01-01         4546.5
3 2019-01-01         4569.5
4 2018-01-01         4387.5


# 4. Statistical Analysis

## Statistical Analysis
- Central Tendency: Mean, Median
- Spread: Standard Deviation, Variance
- Distribution: Skewness, Kurtosis

In [920]:
print("\nStatistical Summary:")
print(data.describe())

# Example: Skewness and Kurtosis
print("\nSkewness:")
print(data.skew())
print("\nKurtosis:")
print(data.kurt())


Statistical Summary:


NameError: name 'data' is not defined

# 5. Data Visualization

## Data Visualization
Visualize key data trends using graphs.


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# 6. Build and Train the ML Model

## Machine Learning Model
- Split the dataset into training and testing sets.
- Build a linear regression model using Scikit-learn.

In [ ]:
# Splitting the Dataset
X = data[['feature1', 'feature2']]  # Replace with actual feature columns
y = data['target']  # Replace with target column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Building the Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
print("\nModel Performance:")
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R-squared (R²):", r2_score(y_test, y_pred))

# 7. Validation

## Validation
Validate the model using cross-validation or other methods.


# 8. Feature Engineering

## Feature Engineering
Consider adding polynomial features or other transformations.


# 9. Evaluation of the Model

## Evaluation
Discuss model results, potential improvements, and transferability.